# RAG Application

## Importing Libraries

In [ ]:
print("Importing libraries...")
#loading and chunking libraries
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os

# vector storage
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

#LECL libraries
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser


# Conversational memory
from langchain_core.messages import HumanMessage, AIMessage
print("libraries successfully imported!")

Importing libraries...
libraries successfully imported!


In [2]:
load_dotenv()
api_key = os.getenv("paid_api")

if not api_key:
    raise ValueError("OPENAI_API_KEY not found in .env file")

print("API key loaded")

API key loaded


## Loading and chunking documents

In [6]:
# data_path = r"C:\Users\owner\Desktop\Files_Deep_Learning\RAG\Project\documents"

data_path = r"C:\Users\owner\Desktop\Files_Deep_Learning\RAG\Project\documents"
# Loading document
documents = []
for file in os.listdir(data_path):
    file_path = os.path.join(data_path, file)

    if file.endswith(".txt"):
        loader = TextLoader(file_path, encoding='utf-8')
        documents.extend(loader.load())

    elif file.endswith(".pdf"):
        loader = PyPDFLoader(file_path)
        documents.extend(loader.load())
print("Documents loaded.")

#Chunking
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap = 50)
chunks = text_splitter.split_documents(documents)
print("document loaded and chunked!")

Documents loaded.
document loaded and chunked!


## Storing embeddings in a vector database

In [7]:



#Creating embeddings and storing
embedding = OpenAIEmbeddings(model= "text-embedding-3-small", openai_api_key=api_key)
vectorstore = Chroma.from_documents(documents=chunks,
                                    embedding=embedding, persist_directory = "./chroma_db")

vectorstore.persist()
print(f"Embeddings created and saved to chroma_db")

Embeddings created and saved to chroma_db


C:\Users\owner\AppData\Local\Temp\ipykernel_22832\3192814298.py:6: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


## RAG Chain with LCEL

In [8]:
# Initialize LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0,openai_api_key=api_key)

# Retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

system_prompt_text = """
You are a personal RAG assistant answering questions strictly from the provided context about Esther Kudoro.

### INSTRUCTIONS:
1. Answer questions about professional experience, skills, repositories, and technical implementation details.
2. Use ONLY the context below. If the answer is not present, say "I do not have that information."
3. ALWAYS cite your sources implicitly by referring to the specific file or section.
4. Format all responses as clean plain text with no markdown or special characters.

### PRIVACY GUARDRAILS (CRITICAL):
You MUST REFUSE to answer questions about the following personal sensitive information, even if it might be present in the context:
- Age
- Date of birth
- Home Address
- Phone number
- Personal Email address
- Any other sensitive personal identifiers

If a user asks for this information, reply EXACTLY with:
"I cannot share personal or sensitive information such as contact details or age. Please ask about her professional experience or projects."


Context:
{context}

Question: {question}

Answer clearly and concisely.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt_text),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")
])

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

## Initialising and activating conversational memory

In [9]:
# RAG CHAIN
from operator import itemgetter
rag_chain = (
    {
        "context": itemgetter("question")
        | retriever
        | format_docs,

        "chat_history": itemgetter("chat_history"),
        "question": itemgetter("question")
    }
    | prompt
    | llm
    | StrOutputParser()
)

print("RAG Chain created!")

RAG Chain created!


## Running the RAG assistant

In [ ]:
chat_history = []
print("Esther Kudoro Personal Assistant \nType either 'exit' or 'quit' to stop the program.")

while True:
    question = input("You: ")
    if question.lower() in ["exit", "quit"]:
        break
    answer = rag_chain.invoke({
        "question": question,
        "chat_history": chat_history
    })

    print(f"\nUser: {question}")
    print(f"Assistant: {answer}\n")

    chat_history.extend([
        HumanMessage(content=question),
        AIMessage(content=answer)
    ])

Esther Kudoro Personal Assistant 
Type either 'exit' or 'quit' to stop the program.

User: who is esther kudoro
Assistant: Esther Kudoro is an Electrical and Electronics Engineer with first-class honors. She has a strong interest in power systems engineering, data analytics, and machine learning. Esther focuses on applying artificial intelligence and data-driven techniques to engineering problems, with professional interests in fault diagnosis, predictive maintenance, and retrieval-augmented generation systems. She possesses technical skills in electrical and electronics engineering, power systems engineering, electrical fault analysis, and has experience with Python for data science and machine learning tasks.


User: what are esther kudoro
Assistant: I do not have that information.


User: what are esther kudoro's core skills
Assistant: Esther Kudoro's core skills include electrical and electronics engineering, power systems engineering, electrical fault analysis, data analysis, feat